In [1]:
from typing import Dict, List, Optional, Any
from pydantic import BaseModel
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import status

from fastapi import Response
from fastapi import Header
from fastapi import Query
from fastapi import Path
from fastapi import Depends

from time import sleep
import nest_asyncio
import uvicorn

In [2]:
import sys
sys.path.append("fastapi_files/")

In [3]:
from fastapi_files.models import Curso

In [4]:
def fake_db():
    try:
        print('Abrindo conexao com o banco de dados...')
        sleep(2)
    finally:
        print('Fechando conexao com o banco de dados')
        sleep(2)

In [5]:
app = FastAPI(title="Api de cursos", version='2.0.1', description='Uma API para estudo do FastAPI')
cursos = {
    1: {
        "titulo": "adifuadfi",
        "aulas": '2323',
        "horas": '21:64'
    },
    2: {
        "titulo": "logica",
        "aulas": '222',
        "horas": '22:12'
    }
}

In [ ]:
@app.get('/cursos', description='Retorna os curso ou uma lista vazia', summary='Retorna todos os cursos', response_model=List[ Curso], response_description('Cursos encontrados com sucesso'))
async def get_cursos(db: Any = Depends(fake_db)):
    return cursos

@app.get('/cursos/{curso_id}')
async def get_curso(curso_id: int = Path(title='ID do curso', description='Deve ser entre 1 e 2', gt=0, lt=3), db: Any = Depends(fake_db)):
    try:
        curso = cursos[curso_id]
        return curso
    except KeyError:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND,
                            detail='Curso nao encontrado')


In [7]:
#passando um status depois de criado com o 201
@app.post('/cursos', status_code=status.HTTP_201_CREATED, response_model=Curso)
async def post_curso(curso: Curso, db: Any = Depends(fake_db)):
    if curso.id not in cursos:
        next_id: int = len(cursos) + 1
        curso.id = next_id
        cursos[next_id] = curso
        return curso
    else:
        raise HTTPException(status_code=status.HTTP_409_CONFLICT, defail=f"Ja existe um curso com o ID {curso.id }")

In [8]:
@app.put('/cursos/{curso_id}')
async def put_curso(curso_id: int, curso: Curso, db: Any = Depends(fake_db)):
    if curso_id in cursos:
        curso.id = curso_id
        cursos[curso_id] = curso
        return curso
    else:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=f'Não foi encontrado o id {curso_id}')

In [9]:
#uma função sempre retorna algo, mesmo que None, sempre retorna algum valor, retorno de um Json ou Response
@app.delete('/cursos/{curso_id}')
async def delete_curso(curso_id: int, db: Any = Depends(fake_db)):
    if curso_id in cursos:
        del cursos[curso_id]
        #return JSONResponse(status_code=status.HTTP_204_NO_CONTENT)
        return Response(status_code=status.HTTP_204_NO_CONTENT)
    else:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=f'Não foi encontrado o id {curso_id}')

In [10]:
@app.get('/calculadora')
async def calcular(a: int=Query(gt=5),b: int=Query(gt=4),c: int=Query(gt=4), x_geek:str = Header(print='isssssssssd'), db: Any = Depends(fake_db)):
    soma = a + b + c
    print(f'X-geek: {x_geek}')
    return {"resultado": soma}

In [11]:
if __name__ == '__main__':
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [5890]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     172.18.0.1:40738 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.18.0.1:40738 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5890]
